# Импорты

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt 
plt.style.use('ggplot')
import seaborn as sns 
sns.set(rc={'figure.figsize':(15, 5)})
import sys
import os
sys.path.append('/home/jovyan') 
from connectors import read_sql_query
from tqdm import tqdm

import sqlalchemy as sa
from scipy.stats import ttest_ind
from datetime import datetime
from typing import Tuple, List, Optional, Dict
import statsmodels.api as sm

## Функции

In [2]:
def get_query_clickhouse(q):
    ch_host = 'rc1b-t40nsk085ch73kmt.mdb.yandexcloud.net'
    ch_cert = '/home/jovyan/CA.pem'
    ch_port = '9440'
    ch_db   = 'analytics'
    ch_user = 'anastasiya_a_kosheleva'
    ch_pass = '092X4oRevXKWBAmB6ZfQBQlV5dNUczTxboSiisfY'

    engine = sa.create_engine(
        f'clickhouse+native://{ch_user}:{ch_pass}@{ch_host}:{ch_port}/{ch_db}?secure=True&ca_certs={ch_cert}'
    )

    return pd.read_sql_query(q, con=engine)

In [3]:
"""
Здесь описаны функции, которыми мы и считаем статтесты. 
Логика вся очень простая и вы всегда можете прочитать докстрингу.
"""

    
def calculate_pvalue(test: str, control: str, 
                     metric: str, group_id: str,
                     df_initial: pd.DataFrame, 
                     metric_type: Optional[str] = 'proportion') -> Tuple[float, float, float]:

    if metric_type == 'ratio':
        df = df_initial[[group_id, 'anonymous_id', metric[0], metric[1]]].dropna()
    else:
        df = df_initial[[group_id, 'anonymous_id', metric]].dropna()
        
    if metric_type == 'average' or metric_type == 'ratio':
        equal_var = False
    else:
        equal_var = True

    if metric_type == 'ratio':
        c_num_values = df[df[group_id] == control][metric[0]].values
        c_denom_values = df[df[group_id] == control][metric[1]].values
        
        t_num_values = df[df[group_id] == test][metric[0]].values
        t_denom_values = df[df[group_id] == test][metric[1]].values
        
        if c_denom_values.sum() == 0:
            print(metric)
        control_mean = c_num_values.sum() / c_denom_values.sum()
        test_mean = t_num_values.sum() / t_denom_values.sum()

        c_values = c_num_values - c_denom_values * control_mean
        t_values = t_num_values - t_denom_values * control_mean
    else:
        c_values = df[df[group_id] == control][metric].values
        t_values = df[df[group_id] == test][metric].values
        
        control_mean = c_values.mean()
        test_mean = t_values.mean()

    return ttest_ind(c_values, t_values, equal_var=equal_var).pvalue, control_mean, test_mean
        
        
def calculate_metrics(test: str, control: str, metrics: List[str],
                      group_id: str, df: pd.DataFrame, metric_type: str,
                      platform: Optional[str] = 'total') -> pd.DataFrame:
    """
    Вычисляет значение p-value
    - для proportion метрик (конверсионных)
        Используется стандартный t-test.
    - для average метрик (для денежных и других средних поюзерных метрик) 
        Используется Welch t-test с поправкой на неравенство дисперсий.
    - для ratio метрик(для глобальных средних метрик или ratio-метрик, например для среднего чека)
        Используется Welch t-test после линеаризации.
        Про линеаризацию тут https://instamart.atlassian.net/wiki/spaces/ANLT/pages/edit-v2/2061107789
    Параметры
    ----------
    test : str
        Название тестовой группы
    control : str
        Название контрольной группы
    metrics : list of str
        Список метрик, для которых нужно посчитать статистику
    group_id : str
        Название колонки, по которой определяется группа
    df : pd.Dataframe
        Датафрейм с данными, уровень группировки - айдишник пользователя. Примеры в запросе для денежных метрик.
    metric_type: str
        Тип метрики. Допустимые значения: 'average', 'ratio', 'proportion'
    """
    if platform == 'total':
        lst = []
        for metric in metrics:
            result =  calculate_pvalue(test, control, metric, group_id, df, metric_type=metric_type)
            lst.append([platform, metric, result[0], result[2], result[1]])
        df = pd.DataFrame(lst, columns=["platform", "metric", "p_value", "test_mean", "control_mean"])
        df["diff"] = df["test_mean"] - df["control_mean"]
        return df
    else:
        lst = []
        for metric in metrics:
            result =  calculate_pvalue(test, control, metric, group_id, df[df['platform'] == platform], metric_type=metric_type)
            lst.append([platform, metric, result[0], result[2], result[1]])
        df = pd.DataFrame(lst, columns=["platform", "metric", "p_value", "test_mean", "control_mean"])
        df["diff"] = df["test_mean"] - df["control_mean"]
        return df

In [241]:
"""
Здесь описаны функции, которыми мы и считаем статтесты. 
Логика вся очень простая и вы всегда можете прочитать докстрингу.
"""

    
def calculate_pvalue(test: str, control: str, 
                     metric: str, group_id: str,
                     df_initial: pd.DataFrame, 
                     metric_type: Optional[str] = 'proportion') -> Tuple[float, float, float]:

    if metric_type == 'ratio':
        df = df_initial[[group_id, 'anonymous_id', metric[0], metric[1]]].dropna()
    else:
        df = df_initial[[group_id, 'anonymous_id', metric]].dropna()
        
    if metric_type == 'average' or metric_type == 'ratio':
        equal_var = False
    else:
        equal_var = True

    if metric_type == 'ratio':
        c_num_values = df[df[group_id] == control][metric[0]].values
        c_denom_values = df[df[group_id] == control][metric[1]].values
        
        t_num_values = df[df[group_id] == test][metric[0]].values
        t_denom_values = df[df[group_id] == test][metric[1]].values
        
        if c_denom_values.sum() == 0:
            print(metric)
        control_mean = c_num_values.sum() / c_denom_values.sum()
        test_mean = t_num_values.sum() / t_denom_values.sum()

        c_values = c_num_values - c_denom_values * control_mean
        t_values = t_num_values - t_denom_values * control_mean
    else:
        c_values = df[df[group_id] == control][metric].values
        t_values = df[df[group_id] == test][metric].values
        
        control_mean = c_values.mean()
        test_mean = t_values.mean()

    return ttest_ind(c_values, t_values, equal_var=equal_var).pvalue, control_mean, test_mean
        
        
def calculate_metrics(test: str, control: str, metrics: List[str],
                      group_id: str, df: pd.DataFrame, metric_type: str,
                      platform: Optional[str] = 'total') -> pd.DataFrame:
    """
    Вычисляет значение p-value
    - для proportion метрик (конверсионных)
        Используется стандартный t-test.
    - для average метрик (для денежных и других средних поюзерных метрик) 
        Используется Welch t-test с поправкой на неравенство дисперсий.
    - для ratio метрик(для глобальных средних метрик или ratio-метрик, например для среднего чека)
        Используется Welch t-test после линеаризации.
        Про линеаризацию тут https://instamart.atlassian.net/wiki/spaces/ANLT/pages/edit-v2/2061107789
    Параметры
    ----------
    test : str
        Название тестовой группы
    control : str
        Название контрольной группы
    metrics : list of str
        Список метрик, для которых нужно посчитать статистику
    group_id : str
        Название колонки, по которой определяется группа
    df : pd.Dataframe
        Датафрейм с данными, уровень группировки - айдишник пользователя. Примеры в запросе для денежных метрик.
    metric_type: str
        Тип метрики. Допустимые значения: 'average', 'ratio', 'proportion'
    """
    if platform == 'total':
        lst = []
        for metric in metrics:
            result =  calculate_pvalue(test, control, metric, group_id, df, metric_type=metric_type)
            lst.append([platform, metric, result[0], result[2], result[1]])
        df = pd.DataFrame(lst, columns=["platform", "metric", "p_value", "test_mean", "control_mean"])
        df["diff"] = df["test_mean"] - df["control_mean"]
        return df
    else:
        lst = []
        for metric in metrics:
            result =  calculate_pvalue(test, control, metric, group_id, df[df['platform'] == platform], metric_type=metric_type)
            lst.append([platform, metric, result[0], result[2], result[1]])
        df = pd.DataFrame(lst, columns=["platform", "metric", "p_value", "test_mean", "control_mean"])
        df["diff"] = df["test_mean"] - df["control_mean"]
        return df


def get_exp_info(exp_id: str) -> Tuple[datetime.date, datetime.date]:
    """
    Выводит мета-информацию по тесту.
    Параметры
    ----------
    exp_id : str
        id теста из A/B-платформы
    
    Возвращает
    -------
    Выводит в аутпут метку теста и названия групп
    Возвращает:
    start_date : str
        Дата начала теста
    end_date : str
        Дата начала теста
    """

    q = f'''
        select 
            at.id as exp_id, 
            at.label as label,
            toDate(at.started_at) as start_date,
            if(
                toDate(at.ended_at) is null,
                today(),
                toDate(at.ended_at) - interval 1 day
            ) as end_date,
            atg.name as name,
            atg.is_reference as is_reference
        from 
            ods.bs_ab_admin__ab_tests at
        inner join 
            ods.bs_ab_admin__ab_test_groups atg on atg.test_id = at.id
        where true
            and toString(at.id) = '{exp_id}'
            and at.dwh_dt = (select max(dwh_dt) from ods.bs_ab_admin__ab_tests)
        '''

    test_info = get_query_clickhouse(q)

    exp_label = test_info['label'][0]
    start_date = test_info['start_date'][0]
    end_date = test_info['end_date'][0]
    
    print(f'Метка теста - {exp_label}')

    groups_dict = test_info[['name', 'is_reference']].T.to_dict('list')
    for group in groups_dict.values():
        if group[1] == 1:
            print(f'Контрольная группа - {group[0]}')
        else:
            print(f'Тестовая группа - {group[0]}')
    return start_date, end_date


def compute_metrics(test_group: str,
                    control_group: str,
                    metrics: Dict[str, List],
                    df: pd.DataFrame) -> pd.DataFrame:
    result = pd.DataFrame(columns = ['platform', 'metric', 'p_value',
                                     'test_mean', 'control_mean', 'diff'])
    
    platforms_list = list(df['platform'].unique())
    if len(platforms_list) > 1:
        platforms_list.append('total')
    
    for platform in platforms_list:
        average = calculate_metrics(
            test_group, control_group, metrics["average"], 'group', df, "average", platform
        )
        proportion = calculate_metrics(
            test_group, control_group, metrics["proportion"], 'group', df, "proportion", platform
        )
        ratio = calculate_metrics(
            test_group, control_group, metrics["ratio"], 'group', df, "ratio", platform
        )
        result = pd.concat([result, average, proportion, ratio]).reset_index(drop=True)
    return result


common_metrics = {
    "average": ['gmv_per_user', 'gmv_per_user_express', 'gmv_per_user_planned',
                'gmv_per_user_pickup', 'gross_profit_per_user', 'gross_profit_per_user_express',
                'gross_profit_per_user_planned', 'gross_profit_per_user_pickup', 'orders', 
                'express_orders', 'planned_orders', 'pickup_orders'],
    "proportion": ['conversion', 'express_conversion', 'planned_conversion', 'pickup_conversion',
                   'conv_to_second_order', 'conv_to_repeated_visit', 'ret_14d'], 
    "ratio": [('gmv_per_user', 'orders'), ('gmv_per_user_express', 'express_orders'), 
              ('gmv_per_user_planned', 'planned_orders'), ('gmv_per_user_pickup', 'pickup_orders')]
}


# MDE-калькулятор, удаление выбросов по метрике
def drop_outliers(df: pd.DataFrame, metric: str) -> pd.DataFrame:
    """
    Удаление выбросов из данных по поданной метрике
    """
    treshhold_ = df[metric][df[metric] > 0].mean() + (df[metric][df[metric] > 0].std() * 9)
    return df[(df[metric] < treshhold_)]


def style_rule(row):
    if row["p_value"] <= 0.01 and row["control_mean"] < row["test_mean"]:
        return ["background-color: #6ccd6c"] + [""]*(len(row)-1)
    elif row["p_value"] <= 0.05 and row["control_mean"] < row["test_mean"]:
        return ["background-color: palegreen"] + [""]*(len(row)-1)
    elif row["p_value"] <= 0.01 and row["control_mean"] > row["test_mean"]:
        return ["background-color: #fa6468"] + [""]*(len(row)-1)
    elif row["p_value"] <= 0.05 and row["control_mean"] > row["test_mean"]:
        return ["background-color: lightsalmon"] + [""]*(len(row)-1)
    else:
        return [""]*len(row)
    
def style_rule_cuped(row):
    if row["pvalue"] <= 0.01 and effect > 0:
        return ["background-color: #6ccd6c"] + [""]*(len(row)-1)
    elif row["pvalue"] <= 0.05 and effect > 0:
        return ["background-color: palegreen"] + [""]*(len(row)-1)
    elif row["pvalue"] <= 0.01 and effect < 0:
        return ["background-color: #fa6468"] + [""]*(len(row)-1)
    elif row["pvalue"] <= 0.05 and effect < 0:
        return ["background-color: lightsalmon"] + [""]*(len(row)-1)
    else:
        return [""]*len(row)

## CUPED

In [5]:
def linearize(data: pd.DataFrame, 
              metric: str,
              numerator: str,
              denominator: str) -> pd.DataFrame:
    data_copy = data.copy()
    k = data_copy.loc[data_copy.treatment==0][numerator].sum() / \
            data_copy.loc[data_copy.treatment==0][denominator].sum()
    data_copy[metric] = data_copy[numerator] - k*data_copy[denominator]
    return data_copy.drop(columns=[denominator, numerator])

In [6]:
def calculate_cuped(pre_data: pd.DataFrame,
                    post_data: pd.DataFrame,
                    group_id: str,
                    test: str,
                    control: str,
                    metric: str,
                    metric_type: str,
                    numerator: Optional[str] = None,
                    denominator: Optional[str] = None) -> None:
    pre_tmp = pre_data.copy()
    post_tmp = post_data.copy()
    
    pre_tmp["treatment"] = np.where(pre_tmp[group_id] == test, 1, 0)
    post_tmp["treatment"] = np.where(post_tmp[group_id] == test, 1, 0)
    
    if metric_type == "ratio":
        lin_effect = post_tmp[post_tmp.treatment == 1][numerator].sum()/post_tmp[post_tmp.treatment == 1][denominator].sum() - post_tmp[post_tmp.treatment == 0][numerator].sum()/post_tmp[post_tmp.treatment == 0][denominator].sum()
        pre_tmp = linearize(pre_tmp, metric, numerator, denominator)
        post_tmp = linearize(post_tmp, metric, numerator, denominator)     
    
    pre_tmp = (pre_tmp.groupby("anonymous_id").agg({metric: "sum"})
                                                  .reset_index()
                                                  .rename({metric: "CUPED_X"}, axis=1))
            
    tmp = post_tmp.merge(pre_tmp, on="anonymous_id", how="left")
    
    # заполнение пропусков
    tmp["missing_CUPED"] = tmp["CUPED_X"].isna().astype(int)
    tmp["CUPED_X"] = tmp["CUPED_X"].fillna(0)
    
    # оценка y_cuped на контроле
    y_control = tmp.query("treatment==0")[metric]
    X_cov_control = tmp.query("treatment==0")[["CUPED_X", "missing_CUPED"]]
    y_hat = sm.OLS(y_control, X_cov_control).fit().predict(tmp[["CUPED_X", "missing_CUPED"]])
    tmp["y_cuped"] = tmp[metric].values - y_hat
    
    # cuped regression
    y_cuped = tmp["y_cuped"]
    X_cuped = tmp["treatment"]
    X_cuped = sm.add_constant(X_cuped)
    model_cuped = sm.OLS(y_cuped, X_cuped)
    results_cuped = model_cuped.fit()
    
    pvalue = results_cuped.pvalues[1]
        
    if metric_type != "ratio":
        effect = results_cuped.params[1]
        e = effect
        # print(f" pvalue for {metric}: {pvalue}. \n estimated effect for {metric}: {effect}")
    elif metric_type == "ratio": 
        e = lin_effect
        # print(f" pvalue for {metric}: {pvalue}. \n estimated effect for {metric}: {lin_effect}")
    return pd.DataFrame([(metric, metric_type, numerator, denominator, pvalue, e)], columns = ['metric','metric_type','numerator','denominator','pvalue','effect'])

In [7]:
def iter_cuped(metrics_dict,
               pre_test_df,
               test_df,
               test,
               control,
               group_id='group'
               ):
    out_df = pd.DataFrame()
    platforms = list(set(test_df.platform.unique().tolist() + ['total']))
    for platform in platforms:
        if platform == "total":
            pre_df = pre_test_df.copy()
            df = test_df.copy()
        else:
            pre_df = pre_test_df.query('platform==@platform').copy()
            df = test_df.query('platform==@platform').copy()
        for metric_type in metrics_dict:
            for metric in metrics_dict[metric_type]:
                if isinstance(metric,str):
                    tmp = calculate_cuped(
                            pre_data=pre_df,
                            post_data=df,
                            group_id=group_id,
                            test=test,
                            control=control,
                            metric=metric,
                            metric_type=metric_type)
                else:
                    tmp = calculate_cuped(
                            pre_data=pre_df,
                            post_data=df,
                            group_id=group_id,
                            test=test,
                            control=control,
                            metric=str(metric),
                            metric_type=metric_type,
                            numerator=metric[0],
                            denominator=metric[1])
                tmp['platform'] = platform
                out_df = pd.concat([out_df,tmp], ignore_index=True)
    return out_df

## Helpers

In [8]:
def create_sbx_table(sandbox_table_name,
                     suffix = '',
                     drop_if_exists=True):
    if suffix:
        sandbox_table_name = sandbox_table_name + '_' + suffix
    
    if not sandbox_table_name.startswith('sandbox'):
            raise Exception(f'"{sandbox_table_name}" is incorrect. sandbox_table_name must start with "sandbox"')
    
    if drop_if_exists:
        read_sql_query(f"""
        drop TABLE if exists {sandbox_table_name}
        """)
        print(f'Table {sandbox_table_name} dropped if existed')
        
    read_sql_query(f"""
        CREATE TABLE {sandbox_table_name}
        (
           `anonymous_id` String,
           `group` String,
           `platform` String,
           `tenant` String,
           `order_id` String,
           `events_dt` Date
        )
        ENGINE = MergeTree
        ORDER BY tuple()
        SETTINGS index_granularity = 8192
        """)
    print(f'Table {sandbox_table_name} created')
    return sandbox_table_name

# Константы

In [9]:
# Задаем параметры экспа, здесь с конкретным примером - ЗДЕСЬ ЗАМЕНИТЬ ID
exp_id = 'f78698a2-10dd-4517-a245-1bf7be32d30a'

In [10]:
start_date, end_date = get_exp_info(exp_id)

Метка теста - tech_oos_badge_test
Тестовая группа - oos_badge_group
Контрольная группа - control


In [11]:
control_group = 'control'
test_group = 'oos_badge_group' #ЗДЕСЬ ЗАМЕНИТЬ ГРУППУ

In [12]:
preend_date = start_date - dt.timedelta(days=1)
prestart_date = preend_date - dt.timedelta(days=(end_date-start_date).days)

In [13]:
oos_stores = read_sql_query(''' select df_oos.store_id as store_id, ret.id as retailer_id
from sandbox.stores_with_out_of_stock_model as df_oos
join int_stores as stores on df_oos.store_id = stores.id
join int_spree_retailers as ret on ret.id = stores.retailer_id
                            ''')
stores_list = oos_stores.store_id.to_list()

In [14]:
event_filter = f"""
'Category Viewed'
and store_id GLOBAL IN (select df_oos.store_id as store_id from sandbox.stores_with_out_of_stock_model as df_oos)
and params['category_id'] GLOBAL NOT IN (select distinct parent_id from ods.product_hub__taxon)
"""

ios_version_filter = f"""
(
  (
    platform = 'ios'
    and (app_version like '7%%' OR app_version like '8%%')
    and (
          (
            toUInt64OrZero(splitByChar('.', app_version)[1]) >= 7 
            and toUInt64OrZero(splitByChar('.', app_version)[2]) >= 16
          )
          or (toUInt64OrZero(app_version) >= 7160000)
          or toUInt64OrZero(splitByChar('.', app_version)[1]) >= 8
        )
  ) 
  OR platform != 'ios'
)
"""

sources_new_app = (
    "Order Completed",
    "Product Added",
    "Login",
    "Shop Selected",
    "Address Change Initiated",
    "Product Viewed",
    "Search Results Viewed",
    "Category Viewed",
    "Shop Selection Started",
    "Pickup Map Opened",
    "Retailer Selection Started",
    "Checkout Sber Spasibo Form Opened",
    "Map Pickup Shop Selected",
    "Checkout Button Clicked",
    "Checkout Loaded",
    "Main Page Viewed",
    "Checkout Delivery Slot Selection Started",
    "Checkout Delivery Slot Selected",
    "Cart Viewed",
    "Search Started",
    "Banner Viewed",
    "Recommendation Block View",
    "Product Category Viewed",
    "Catalogue Tab Clicked",
    "Onboarding Banner Viewed",
    "Order Info Viewed",
    "Product Sort Initiated",
    "Product Filter Initiated",
    "Product Sorted",
    "Address Selected",
    "Address Search Opened",
    "Address Suggest Clicked",
    "Redirected From SBOL",
    "Sberid Authorization Prompt Shown",
    "Map Pickup Button Clicked",
    "Age Confirmation Viewed",
    "Favorites Tab Clicked",
    "Address Change Clicked",
    "Retailer Selected"
)

sources_web = (
    "Order Completed",
    "Checkout Button Clicked",
    "Landing Viewed",
    "Product Added",
    "Main Page Viewed",
    "Suggester Viewed",
    "Main Page Viewed",
    "Shop Selected",
    "Product Viewed",
    "Recommendation Product View",
    "Search Results Viewed",
    "Category Viewed",
    "Shop Selection Started",
    "Pickup Map Opened",
    "Cart Viewed",
    "Order Merged",
    "Checkout Loaded",
    "Pre Replacements Product Selected",
    "Pre Replacements Product All Choosed",
    "Address Change Initiated",
    "Retailer Shipping Method Clicked",
    "Button Find Stores Clicked",
    "Favourites Tab Clicked",
    "Checkout Slot Selection Started",
    "Checkout Delivery Slot Selected",
    "Search Started",
    "Search Completed",
    "Search Suggest Clicked",
    "Suggester Viewed",
    "Banner Viewed",
    "Recommendation Block View",
    "Product Category Viewed",
    "Add To Cart Clicked",
    "Product Sort Initiated",
    "Product Sorted",
    "Cart Params Loaded",
    "Address Suggest Clicked",
    "Alcohol Pickup Button Clicked",
    "Only Pickup Popup Viewed",
    "Retailer Selected"
)

In [157]:
if event_filter:
    filter_events = f"""filter_events as (
    select
        anonymous_id,
        toDate(min(ts)) as event_dt
    from event.new_app
    where 1=1
        and event = {event_filter}
        and dwh_dt between start_date and end_date
        and toDate(ts) between start_date and end_date
        and {ios_version_filter}
    group by anonymous_id
    
    UNION ALL 

    select
        toString(anonymous_id) as anonymous_id,
        toDate(min(ts)) as event_dt
    from event.web
    where 1=1
        and event = {event_filter}
        and dwh_dt between start_date and end_date
        and toDate(ts) between start_date and end_date
    group by anonymous_id
    ),"""
    
    anon_in = """and anonymous_id global in 
    (
      select anonymous_id 
      from filter_events
    )"""
    
    join_filter = "inner join filter_events on ab_groups.anonymous_id = filter_events.anonymous_id"
    
    date_filter = "and filter_events.event_dt <= events_dt"
    
else:
    filter_events, anon_in, join_filter, date_filter = "", "", "", ""

In [217]:
create_sbx_table('sandbox_research.ab_tovara_ostalos_malo',
                             suffix = '',
                             drop_if_exists=True)

Table sandbox_research.ab_tovara_ostalos_malo dropped if existed
Table sandbox_research.ab_tovara_ostalos_malo created


'sandbox_research.ab_tovara_ostalos_malo'

In [218]:
sbx_table = 'sandbox_research.ab_tovara_ostalos_malo'

In [219]:
read_sql_query(f"""
insert into {sbx_table}
with 
    toDate('{start_date}') as start_date,
    toDate('{end_date}') as end_date,
    toString('{exp_id}') as exp_id,    

events as (
    select 
        toString(anonymous_id) as anonymous_id,
        toDate(ts) as dt,
        'web' as platform,
        ifNull(NullIf(tenant, ''), 'sbermarket') as tenant
    from event.web
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and not_bot
        and event in {sources_web}
    group by anonymous_id, dt, platform, tenant
    
    union all
    
    select 
        anonymous_id,
        toDate(ts) as dt,
        toString(platform) as platform,
        'sbermarket' as tenant
    from event.new_app
    where 1=1
        and toDate(ts) between start_date and end_date
        and dwh_dt between start_date and end_date
        and event in {sources_new_app}
        and {ios_version_filter}
    group by anonymous_id, dt, platform, tenant
),

{filter_events}

ab_groups as (
    select 
        anonymous_id,
        group,
        date_msk as dt
    from cdm.ab__groups__anon 
    where 1=1 
        and test_id = exp_id
        {anon_in}
),

client_id_x_order_id as (
    select distinct
        date_msk as date,
        uuid,
        anonymous_id,
        if(platform in ('desktop', 'mobile'), 'web', platform) as platform,
        tenant,
        order_id
    from 
        cdm.ab__client_id_x_order_id
    where true 
        and date_msk between start_date and end_date
),

ab_groups_x_events as (
    select distinct
        ab_groups.anonymous_id as anonymous_id,
        group,
        events.platform as platform,
        events.tenant as tenant,
        client_id_x_order_id.order_id as order_id,
        events.dt as events_dt
    from 
        ab_groups
        
        {join_filter}
    
        inner join events 
            on ab_groups.anonymous_id = events.anonymous_id 
        
        left join client_id_x_order_id
            on ab_groups.anonymous_id = client_id_x_order_id.anonymous_id
            and events.dt = client_id_x_order_id.date
            and events.platform = client_id_x_order_id.platform
            and events.tenant = client_id_x_order_id.tenant
            
    where true
      and ab_groups.dt <= events_dt
      {date_filter}
)

select * FROM ab_groups_x_events
""")

""


# Базовые метрики

In [45]:
def get_general_metrics_within_retlist(start_date, end_date, sbx_table):
    q = f"""
    with 
    toDate('{start_date}') as start_date,
    toDate('{end_date}') as end_date, 

    ab_groups_x_events AS (
        SELECT *
        FROM {sbx_table}
    ),

    late_cancelled_orders as (
     select 
           stateful_id as order_id,
           minIf(toDate(created_at), next_state = 'complete') as completed_at,
           minIf(toDate(created_at), next_state = 'canceled') as canceled_at,
           if((canceled_at > completed_at + interval 2 day or canceled_at > end_date), 1, 0) as late_cancel_flg
      from (
      select stateful_type,
            created_at,
            next_state,
            stateful_id
        from analytics.int_spree_state_changes  
       where stateful_type = 'Spree::Order'
         and toDate(created_at) between start_date and end_date
         and next_state in ('complete', 'canceled')
        order by created_at) issc
     group by stateful_id
    having completed_at is not null
       and late_cancel_flg
    ),

    financial_measures as (
        select 
            uuid,
            order_number,
            toDate(completed_at) as completed_dt,
            max(shipped_at) as shipped_at,
            sumIf(1, type_store_delivery == 'planned') as planned_store_flg,
            sumIf(1, type_store_delivery == 'on_demand') as express_store_flg,
            sumIf(1, type_delivery=='asap') as express_flg,
            sumIf(1, type_delivery=='planned') as planned_flg,
            sumIf(1, type_delivery=='pickup') as pickup_flg,
            sumIf(gmv_service_fee_net_promo, type_store_delivery == 'on_demand') +
                sumIf(gmv_goods_net_promo, type_store_delivery == 'on_demand') as gmv_net_of_promo_express,
            sumIf(gmv_service_fee_net_promo, type_store_delivery == 'planned') +
                sumIf(gmv_goods_net_promo, type_store_delivery == 'planned') as gmv_net_of_promo_planned,
            sumIf(gmv_service_fee_net_promo, type_delivery = 'pickup') +
                sumIf(gmv_goods_net_promo, type_delivery = 'pickup') as gmv_net_of_promo_pickup,
            sum(gmv_service_fee_net_promo) + sum(gmv_goods_net_promo) as gmv_net_of_promo,
            sumIf(gross_profit, type_delivery = 'asap') as gross_profit_express,
            sumIf(gross_profit, type_delivery = 'planned') as gross_profit_planned,
            sumIf(gross_profit, type_delivery = 'pickup') as gross_profit_pickup,
            sumIf(gross_profit, type_store_delivery = 'on_demand') as gross_profit_express_store,
            sumIf(gross_profit, type_store_delivery = 'planned') as gross_profit_planned_store,
            sum(gross_profit) as gross_profit_full

        from gp_rep.rep__bi_shipment
        where 1=1
            and store_id IN {tuple(stores_list)}
            and toDate(completed_at) between start_date and end_date
            and user_id is not null
            and order_state in ('complete', 'canceled')
            and b2b_order_company_flg != 1
            and b2b_measure = 0
            and order_id not in (select order_id from late_cancelled_orders)
            and order_number not in 
            (
                select order_number
                from gp_rep.rep__bi_shipment
                where 1=1
                    and toDate(completed_at) between start_date and end_date + interval 14 day
                    and owner_type = 'ServiceAccount'
            )
        group by dictGet('analytics.spree_users_dict', 'uuid', toUInt64(user_id)) as uuid, order_number, completed_at
        )

    select 
        group,
        anonymous_id,
        platform,
        tenant,
        uniqExactIf(order_id, uuid != '' and completed_dt = events_dt) as orders, -- это значение может быть использовано как знаменатель для ratio метрики среднего чека
        uniqExactIf(order_id, uuid != '' and express_flg > 0 and completed_dt = events_dt) as express_orders,
        uniqExactIf(order_id, uuid != '' and planned_flg > 0 and completed_dt = events_dt) as planned_orders,
        uniqExactIf(order_id, uuid != '' and express_store_flg > 0 and completed_dt = events_dt) as express_store_orders,
        uniqExactIf(order_id, uuid != '' and planned_store_flg > 0 and completed_dt = events_dt) as planned_store_orders,
        uniqExactIf(order_id, uuid != '' and pickup_flg > 0 and completed_dt = events_dt) as pickup_orders,

        max(if(uuid != '' and completed_dt = events_dt, 1, 0)) as conversion,
        max(if(uuid != '' and express_flg > 0 and completed_dt = events_dt, 1, 0)) as express_conversion,
        max(if(uuid != '' and planned_flg > 0 and completed_dt = events_dt, 1, 0)) as planned_conversion,
        max(if(uuid != '' and pickup_flg > 0 and completed_dt = events_dt, 1, 0)) as pickup_conversion,

        sumIf(gmv_net_of_promo, completed_dt = events_dt) as gmv_per_user, -- это значение может быть использовано как числитель для ratio метрики среднего чека
        sumIf(gmv_net_of_promo_express, completed_dt = events_dt) as gmv_per_user_express,
        sumIf(gmv_net_of_promo_planned, planned_flg > 0 and completed_dt = events_dt) as gmv_per_user_planned,
        sumIf(gmv_net_of_promo_pickup, pickup_flg > 0 and completed_dt = events_dt) as gmv_per_user_pickup,

        sumIf(gross_profit_full, completed_dt = events_dt) as gross_profit_per_user,
        sumIf(gross_profit_express, completed_dt = events_dt) as gross_profit_per_user_express,
        sumIf(gross_profit_planned, completed_dt = events_dt) as gross_profit_per_user_planned,
        sumIf(gross_profit_pickup, completed_dt = events_dt) as gross_profit_per_user_pickup,
        sumIf(gross_profit_express_store, express_store_flg > 0 and completed_dt = events_dt) as gross_profit_per_user_express_store,
        sumIf(gross_profit_planned_store, planned_store_flg > 0 and completed_dt = events_dt) as gross_profit_per_user_planned_store,

        minIf(completed_dt, completed_dt = events_dt) as first_order_date,
        if(
            first_order_date is null, 
            null, 
            arrayExists(
                elem -> assumeNotNull(elem <= toDate(end_date) and toDate(elem) > first_order_date), 
                groupArray(toDate(completed_dt))
            )
        ) as conv_to_second_order,

        if(
            first_order_date is null, 
            null, 
            arrayExists(
                elem -> assumeNotNull(elem <= toDate(end_date) and toDate(elem) > first_order_date), 
                groupArray(toDate(events_dt))
            )
        ) as conv_to_repeated_visit,    
        arrayExists(elem -> assumeNotNull(elem <= toDate(first_order_date + INTERVAL 14 DAY) and 
            elem > first_order_date), groupArray(toDate(shipped_at))) as ret_14d
    from ab_groups_x_events
    left join (select * from financial_measures 
                where completed_dt >= start_date and completed_dt <= end_date + interval 14 day) financial_measures
        on order_id = order_number
    where tenant = 'sbermarket'
    group by group, anonymous_id, platform, tenant
    """
    return read_sql_query(q)

In [46]:
general_metrics_within_retlist = get_general_metrics_within_retlist(start_date, end_date, sbx_table)

In [243]:
drop_outliers(general_metrics_within_retlist)

TypeError: drop_outliers() missing 1 required positional argument: 'metric'

In [48]:
ab_groups = drop_outliers(general_metrics_within_retlist)[['group','anonymous_id','platform']]

In [78]:
general_metrics_within_retlist_pre = get_general_metrics_within_retlist(prestart_date, preend_date, sbx_table)

In [79]:
general_metrics_within_retlist_result_cuped = iter_cuped(
    common_metrics,
    drop_outliers(general_metrics_within_retlist_pre),
    drop_outliers(general_metrics_within_retlist),
    test = test_group,
    control = control_group)

/tmp/ipykernel_144/3164479529.py:6: RuntimeWarning: invalid value encountered in double_scalars
  k = data_copy.loc[data_copy.treatment==0][numerator].sum() / \
/tmp/ipykernel_144/3164479529.py:6: RuntimeWarning: invalid value encountered in double_scalars
  k = data_copy.loc[data_copy.treatment==0][numerator].sum() / \
/tmp/ipykernel_144/3164479529.py:6: RuntimeWarning: invalid value encountered in double_scalars
  k = data_copy.loc[data_copy.treatment==0][numerator].sum() / \
/tmp/ipykernel_144/3164479529.py:6: RuntimeWarning: invalid value encountered in double_scalars
  k = data_copy.loc[data_copy.treatment==0][numerator].sum() / \
/tmp/ipykernel_144/3164479529.py:6: RuntimeWarning: invalid value encountered in double_scalars
  k = data_copy.loc[data_copy.treatment==0][numerator].sum() / \
/tmp/ipykernel_144/3164479529.py:6: RuntimeWarning: invalid value encountered in double_scalars
  k = data_copy.loc[data_copy.treatment==0][numerator].sum() / \
/tmp/ipykernel_144/3164479529.py:6

In [80]:
general_metrics_within_retlist_result_cuped.style.apply(style_rule_cuped, axis=1)

,metric,metric_type,numerator,denominator,pvalue,effect,platform
0,gmv_per_user,average,None,None,0.647579,33.244078,ios
1,gmv_per_user_express,average,None,None,0.657555,-15.474578,ios
2,gmv_per_user_planned,average,None,None,0.418702,50.436407,ios
3,gmv_per_user_pickup,average,None,None,0.664251,1.625713,ios
4,gross_profit_per_user,average,None,None,0.300738,10.554049,ios
5,gross_profit_per_user_express,average,None,None,0.996852,-0.016338,ios
6,gross_profit_per_user_planned,average,None,None,0.276879,9.985804,ios
7,gross_profit_per_user_pickup,average,None,None,0.386550,0.371486,ios
8,orders,average,None,None,0.645772,0.013553,ios
9,express_orders,average,None,None,0.689207,0.008067,ios


In [21]:
result_basic_metrics = compute_metrics(
    test_group=test_group, control_group=control_group, metrics=common_metrics, df=drop_outliers(general_metrics_within_retlist)
)

In [22]:
result_basic_metrics.style.apply(style_rule, axis=1)

,platform,metric,p_value,test_mean,control_mean,diff
0,android,gmv_per_user,0.778762,1219.662606,1228.512957,-8.850351
1,android,gmv_per_user_express,0.100712,153.859278,166.940729,-13.081452
2,android,gmv_per_user_planned,0.498160,709.417258,693.936539,15.480718
3,android,gmv_per_user_pickup,0.236892,342.457337,363.418331,-20.960995
4,android,gross_profit_per_user,0.185209,56.336500,60.650053,-4.313553
5,android,gross_profit_per_user_express,0.473632,6.429109,7.011775,-0.582667
6,android,gross_profit_per_user_planned,0.210911,48.958835,52.508804,-3.549969
7,android,gross_profit_per_user_pickup,0.967600,1.563487,1.516550,0.046938
8,android,orders,0.918341,0.514487,0.515625,-0.001138
9,android,express_orders,0.353818,0.075386,0.078984,-0.003598


# Кастомные метрики

### CR Product Rendered → Product Added

In [220]:
#метрики из событийки - product rendered
def get_custom_metrics_product_rendered(
                                 start_date, 
                                 end_date, 
                                 sbx_table,
                                 received_at_window=3):
    q = f"""
    with 
        toDate('{start_date}') as start_date,
        toDate('{end_date}') as end_date,

    ab_groups_x_events AS (
            SELECT *
            FROM {sbx_table}
        ),

t1 as (
    select 
           anonymous_id,
           toDate(ts) as dt,
           toString(platform) as platform,
           order_id,
           event
      from event.new_app
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event in ('Product Rendered') 
       and source in ('category', 'search', 'search_multiretail')
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (52, 1087)
       and {ios_version_filter} 

    UNION ALL 

    select 
           toString(anonymous_id) AS anonymous_id,
           toDate(ts) as dt,
           'web' as platform,
           order_id,
           event
      from event.web
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event in ('Product Rendered') 
       and source in ('category', 'search', 'search_multiretail')
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (52, 1087)
    ),
    
    t2 as (
    select
        anonymous_id,
        dt,
        platform,
        order_id,
        countIf(event = 'Product Rendered') as product_rendered_count
    from t1
    group by 1,2,3,4
    )

   select 
         ab_groups_x_events.platform AS platform,
         ab_groups_x_events.anonymous_id AS anonymous_id,
         case when sum(product_rendered_count) > 0 then 1 else 0 end as product_rendered_user
      from ab_groups_x_events 
             join t2 ON t2.order_id = ab_groups_x_events.order_id
                    and t2.dt = ab_groups_x_events.events_dt
                    and t2.anonymous_id = ab_groups_x_events.anonymous_id
                    and toString(t2.platform) = toString(ab_groups_x_events.platform)
     GROUP BY 1,2
    SETTINGS enable_optimize_predicate_expression=0
    """
    return read_sql_query(q)

In [221]:
custom_metrics_product_rendered_data = get_custom_metrics_product_rendered(start_date, end_date, sbx_table)

In [222]:
custom_metrics_product_rendered_data

,platform,anonymous_id,product_rendered_user
0,ios,ca580869-e6b2-42c8-b106-8ea49d170284,1
1,android,41257f261c1c415e,1
2,android,d4028d09f1056cfc,1
3,android,4c91a5e6843f6290,1
4,android,ad5d163d0aea690a,1
...,...,...,...
13557,android,ecae6a1c3d0bf1ac,1
13558,android,8021aaa64dab6b59,1
13559,android,b16f9fa55260cd39,1
13560,android,e74cfc3e8650bf62,1


In [223]:
#метрики из событийки - product added и order completed
def get_custom_metrics_other(
                                 start_date, 
                                 end_date, 
                                 sbx_table,
                                 received_at_window=3):
    q = f"""
    with 
        toDate('{start_date}') as start_date,
        toDate('{end_date}') as end_date,
        toString('{exp_id}') as exp_id,    

    ab_groups_x_events AS (
            SELECT *
            FROM {sbx_table}
        ),

t1 as (
    select 
           anonymous_id,
           toDate(ts) as dt,
           toString(platform) as platform,
           order_id,
           event,
           simpleJSONExtractUInt(params['badges'], 'badge_id') as badge_id
      from event.new_app
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and (
       (event in ('Product Added') and source in ('category', 'search', 'search_multiretail'))
       or (event in ('Order Completed'))
       )
       and {ios_version_filter} 

    UNION ALL 

    select 
           toString(anonymous_id) AS anonymous_id,
           toDate(ts) as dt,
           'web' as platform,
           order_id,
           event,
           simpleJSONExtractUInt(params['badges'], 'badge_id') as badge_id
      from event.web
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and (
       (event in ('Product Added') and source in ('category', 'search', 'search_multiretail'))
       or (event in ('Order Completed'))
       )
    ),
    
    t2 as (
    select
        anonymous_id,
        dt,
        platform,
        order_id,
        countIf(event = 'Product Added' and badge_id in (52, 1087)) as product_added_badge,
        countIf(event = 'Product Added' and badge_id not in (52, 1087)) as product_added_no_badge,
        countIf(event = 'Order Completed') as order_completed_count
    from t1
    group by 1,2,3,4
    )

   select 
         ab_groups_x_events.platform AS platform,
         ab_groups_x_events.anonymous_id AS anonymous_id,
         sum(product_added_badge) as product_added_badge_ratio,
         case when sum(product_added_badge) > 0 then 1 else 0 end as product_added_badge_user,
         case when sum(product_added_no_badge) > 0 then 1 else 0 end as product_added_no_badge_user,
         case when sum(order_completed_count) > 0 then 1 else 0 end as order_completed_count_user
      from ab_groups_x_events 
            join t2 ON t2.order_id = ab_groups_x_events.order_id
                    and t2.dt = ab_groups_x_events.events_dt
                    and t2.anonymous_id = ab_groups_x_events.anonymous_id
                    and toString(t2.platform) = toString(ab_groups_x_events.platform)
     GROUP BY 1,2
    SETTINGS enable_optimize_predicate_expression=0
    """
    return read_sql_query(q)

In [224]:
custom_metrics_other_data = get_custom_metrics_other(start_date, end_date, sbx_table)

In [225]:
custom_metrics_other_data

,platform,anonymous_id,product_added_badge_ratio,product_added_badge_user,product_added_no_badge_user,order_completed_count_user
0,android,2e6496e25b20731c,0,0,1,0
1,android,aabf40554361e674,0,0,1,1
2,android,1b8cfc958e51b144,0,0,1,0
3,android,41257f261c1c415e,0,0,1,0
4,android,8840cf8bb54b2db1,0,0,1,0
...,...,...,...,...,...,...
24023,android,b16f9fa55260cd39,0,0,1,0
24024,android,8e1d926a95a7c66b,0,0,1,1
24025,android,58c0ee9745d9532d,0,0,1,0
24026,android,e74cfc3e8650bf62,0,0,1,1


In [226]:
custom_metrics_full_data = custom_metrics_product_rendered_data.merge(custom_metrics_other_data, \
                                                          how = 'left', on = ['anonymous_id', 'platform'])
custom_metrics_full_data = (
    custom_metrics_full_data
    .merge(ab_groups, on=['platform','anonymous_id'])
)

In [227]:
custom_metrics_full= \
{'average':[("product_added_badge_ratio")],
 'proportion': [("product_added_badge_user"), ("product_added_no_badge_user"), ("order_completed_count_user")],
 'ratio': [
 ]}

In [228]:
custom_metrics_full_result = compute_metrics(
    test_group=test_group, control_group=control_group, metrics=custom_metrics_full, \
      df=custom_metrics_full_data
)
custom_metrics_full_result.style.apply(style_rule, axis=1)

,platform,metric,p_value,test_mean,control_mean,diff
0,ios,product_added_badge_ratio,0.673506,0.516129,0.480278,0.035851
1,ios,product_added_badge_user,0.750700,0.238710,0.229698,0.009011
2,ios,product_added_no_badge_user,0.608529,0.995699,0.997680,-0.001981
3,ios,order_completed_count_user,0.164259,0.501075,0.547564,-0.046489
4,android,product_added_badge_ratio,0.028946,0.460573,0.400702,0.059871
5,android,product_added_badge_user,0.109684,0.198899,0.186218,0.012680
6,android,product_added_no_badge_user,0.903974,0.994273,0.994088,0.000185
7,android,order_completed_count_user,0.233948,0.592070,0.580270,0.011801
8,total,product_added_badge_ratio,0.023109,0.465734,0.406571,0.059163
9,total,product_added_badge_user,0.084615,0.202597,0.189425,0.013172


In [229]:
def get_rc_share_in_orders(
                                 start_date, 
                                 end_date, 
                                 sbx_table,
                                 received_at_window=3):
    q = f"""
    with 
        toDate('{start_date}') as start_date,
        toDate('{end_date}') as end_date,
        toString('{exp_id}') as exp_id,    

    ab_groups_x_events AS (
            SELECT *
            FROM {sbx_table}
        ),

    t1 as (

    select 
           anonymous_id,
           toDate(ts) as dt,
           toString(platform) as platform,
           order_id
      from event.new_app
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Added'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (52, 1087)
       and source in ('category', 'search', 'search_multiretail')
       and {ios_version_filter}
     group by 1,2,3,4

    UNION ALL 

    select 
           toString(anonymous_id) AS anonymous_id,
           toDate(ts) as dt,
           'web' as platform,
           order_id
      from event.web
     where 1 = 1
       and toDate(ts) between start_date and end_date
       and dwh_dt between start_date and end_date + {received_at_window}
       AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
       and event = 'Product Added'
       and simpleJSONExtractUInt(params['badges'], 'badge_id') in (52, 1087)
       and source in ('category', 'search', 'search_multiretail')
     group by 1,2,3,4
    ),
    
    li AS (
    SELECT shipment_id,
           line_item_id,
           retailer_sku,
           sku as product_sku,
           retailer_name,
           li_deleted_at,
           order_number,
           toDate(order_completed_at) as order_completed_at
      FROM line_items
    where 1 = 1
            and toDate(order_completed_at) between start_date and end_date
            and toDate(li_created_at) BETWEEN start_date-60 AND end_date
            AND toUInt64OrZero(toString(store_id)) global IN (select store_id from sandbox.stores_with_out_of_stock_model)
            AND shipment_state = 'shipped'
            AND li_created_at is not null
            AND not (li_deleted_at is not null and assembly_issue is null)
            AND sku not in (select sku from sandbox.damage_packages)
            AND retailer_sku <> '4000103813'
            AND order_number global in (select order_id from t1)
    ),

    repl AS (
    SELECT from_item_id,
           to_item_id
      FROM int_line_item_replacements
     WHERE shipment_id IN (SELECT shipment_id FROM li)
    ),

    canc AS (
    SELECT *
      FROM int_line_item_cancellations
     WHERE shipment_id IN (SELECT shipment_id FROM li)
    ),

    prep AS (
    SELECT  order_number,
            retailer_sku,
            product_sku,
            order_completed_at,
            multiIf(canc.line_item_id >0, 'cancelled', repl.from_item_id>0, 'replaced', 'ok') AS rc_status,
            if(li.line_item_id IN (SELECT to_item_id FROM repl), 1, 0) AS is_replacement,
            multiIf(li_deleted_at IS NOT NULL, 1, 0) AS is_deleted
       FROM li AS li
            LEFT JOIN repl ON repl.from_item_id = toUInt64(li.line_item_id)
            LEFT JOIN canc ON canc.line_item_id = toUInt64(li.line_item_id)
    ),
    
    t2 as (
    SELECT  order_number, 
    order_completed_at,
        sumIf(1, rc_status != 'ok') AS rc_num,
        sum(multiIf((rc_status != 'ok') OR (is_deleted = 0 AND is_replacement = 0), 1, 0)) AS rc_denom
    FROM prep 
    GROUP BY 1,2)

    select 
           ab_groups_x_events.platform AS platform,
           ab_groups_x_events.anonymous_id AS anonymous_id,
           SUM(rc_num) AS rc_num,
           SUM(rc_denom) AS rc_denom
      from ab_groups_x_events a 
           join t2 b ON a.order_id = toString(b.order_number) and a.events_dt = t2.order_completed_at
     GROUP BY 1,2

    SETTINGS enable_optimize_predicate_expression=0
    """
    return read_sql_query(q)

In [230]:
rc_share_in_orders_data = get_rc_share_in_orders(start_date, end_date, sbx_table)

In [231]:
rc_share_in_orders_data

,platform,anonymous_id,rc_num,rc_denom
0,android,ad5d163d0aea690a,2,14
1,android,0a826988b7a543d7,0,18
2,android,9f970e9c801b8ec8,4,25
3,android,7ea99492b461d7e8,3,34
4,ios,f4094709-87d2-4f2d-bbf5-dcaf852537fa,1,23
...,...,...,...,...
1398,android,eaea81b8682f89e3,3,33
1399,android,bb10b7ef65f9cfac,1,13
1400,ios,ae775dce-5dd7-4853-9e22-c78763c9309e,0,22
1401,android,d39d4665c28ab80c,1,12


In [ ]:
rc_share_in_orders_data_pre = get_rc_share_in_orders(prestart_date, preend_date, sbx_table)

In [232]:
rc_share_in_orders_data = (
    rc_share_in_orders_data
    .merge(ab_groups, on=['platform','anonymous_id'])
)

# rc_share_in_orders_data_pre = (
#     rc_share_in_orders_data_pre
#     .merge(ab_groups, on=['platform','anonymous_id'])
# )

In [233]:
rc_share_in_orders_data_metrics = \
{'average':[  ("rc_num") ],
 'proportion': [],
 'ratio': [("rc_num", "rc_denom")]}

In [ ]:
# rc_share_in_orders_data_result_cuped = iter_cuped(
#     rc_share_in_orders_data_metrics,
#     rc_share_in_orders_data_pre,
#     rc_share_in_orders_data,
#     test = test_group,
#     control = control_group)

In [234]:
rc_share_in_orders_data_result = compute_metrics(
    test_group=test_group, control_group=control_group, metrics=rc_share_in_orders_data_metrics, \
      df=rc_share_in_orders_data
)

In [235]:
rc_share_in_orders_data_result.style.apply(style_rule, axis=1)

,platform,metric,p_value,test_mean,control_mean,diff
0,android,rc_num,0.140891,1.433673,1.287988,0.145685
1,android,"('rc_num', 'rc_denom')",0.320359,0.071532,0.067486,0.004046
2,ios,rc_num,0.579389,0.950000,1.071429,-0.121429
3,ios,"('rc_num', 'rc_denom')",0.734381,0.050398,0.054299,-0.003901
4,total,rc_num,0.205761,1.388889,1.271754,0.117135
5,total,"('rc_num', 'rc_denom')",0.407354,0.069681,0.066466,0.003215


In [236]:
df_list = []
df_list.extend([result_basic_metrics, custom_metrics_full_result, rc_share_in_orders_data_result])
final_result_table = pd.concat(df_list)

In [237]:
final_result_table['diff_relative'] = final_result_table['test_mean']/final_result_table['control_mean']*100-100

In [238]:
final_result_table

,platform,metric,p_value,test_mean,control_mean,diff,diff_relative
0,android,gmv_per_user,0.778762,1219.662606,1228.512957,-8.850351,-0.720412
1,android,gmv_per_user_express,0.100712,153.859278,166.940729,-13.081452,-7.835986
2,android,gmv_per_user_planned,0.498160,709.417258,693.936539,15.480718,2.230855
3,android,gmv_per_user_pickup,0.236892,342.457337,363.418331,-20.960995,-5.767732
4,android,gross_profit_per_user,0.185209,56.336500,60.650053,-4.313553,-7.112200
...,...,...,...,...,...,...,...
1,android,"(rc_num, rc_denom)",0.320359,0.071532,0.067486,0.004046,5.995376
2,ios,rc_num,0.579389,0.950000,1.071429,-0.121429,-11.333333
3,ios,"(rc_num, rc_denom)",0.734381,0.050398,0.054299,-0.003901,-7.183908
4,total,rc_num,0.205761,1.388889,1.271754,0.117135,9.210526


In [240]:
final_result_table.to_excel('oos_tovara_ostalos_malo_v2_result.xlsx', index = False)